<a href="https://colab.research.google.com/github/ValentinCord/HandsOnAI_2/blob/main/NLP_Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!/opt/bin/nvidia-smi
!rm -rf sample_data

/bin/bash: /opt/bin/nvidia-smi: No such file or directory


In [2]:
!pip3 install transformers
!pip3 install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 4.7 MB/s 
     |████████████████████████████████| 7.6 MB 32.3 MB/s 
     |████████████████████████████████| 182 kB 15.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 5.1 MB/s 
     |████████████████████████████████| 212 kB 59.6 MB/s 
     |████████████████████████████████| 132 kB 58.6 MB/s 
     |████████████████████████████████| 127 kB 58.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [35]:
# basics 
import os
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score

# transformers 
from datasets import load_dataset
from transformers import pipeline, AutoTokenizer
from transformers import CamembertModel, CamembertTokenizer
from transformers import DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

# plot 
import matplotlib.pyplot as plt 
import seaborn as sns 

In [4]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# Lecture et analyse pandas

In [10]:
train_path = '/content/drive/MyDrive/HandOnAI_2_NLP/fake_train.csv'
test_path = '/content/drive/MyDrive/HandOnAI_2_NLP/fake_test.csv'

In [11]:
df_train = pd.read_csv(train_path)
df_test = pd.read_csv(test_path)

In [26]:
df_train.head()

,Unnamed: 0,data,label,target_name
0,0,Au 22e jour consécutif de la grève reconductib...,0,News
1,1,"Depuis plusieurs mois, les initiatives des che...",0,News
2,2,Google vient d'introduire une mise à jour de...,0,News
3,3,Portrait. Chacun s’empresse autour d’elle tan...,0,News
4,4,« Il n’y a pas de risque de pénurie » de carbu...,0,News


In [24]:
df_train['label'].value_counts()

0    816
1    642
Name: label, dtype: int64

In [25]:
df_train['target_name'].value_counts()

News         816
Fake News    642
Name: target_name, dtype: int64

# Lecture en Dataset

In [30]:
ds = load_dataset("csv", data_files=train_path)
ds_test = load_dataset("csv", data_files=test_path, split='train')

  0%|          | 0/1 [00:00<?, ?it/s]

In [31]:
ds = ds['train'].train_test_split(test_size = 0.1)

In [32]:
ds = ds.remove_columns("Unnamed: 0")
ds = ds.remove_columns("target_name")

ds_test = ds_test.remove_columns('Unnamed: 0')
ds_test = ds_test.remove_columns('target_name')

In [34]:
model_id = 'camembert-base'

tokenizer = CamembertTokenizer.from_pretrained("camembert-base")
camembert = CamembertModel.from_pretrained("camembert-base")

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [39]:
print( df_train['data'][0])
print( tokenizer.tokenize(df_train['data'][0]))
print( tokenizer.convert_tokens_to_ids(tokenizer.tokenize(df_train['data'][0])))

Au 22e jour consécutif de la grève reconductible contre la réforme des retraites (soit autant de jours que le long conflit de 1995), le taux de grévistes à la SNCF a légèrement baissé à 9,6 % jeudi matin, contre 9,8 % mardi, selon la direction du groupe public. Parmi les personnels indispensables à la circulation des trains, moins de la moitié des conducteurs (42,1 %) et le quart (24 %) des contrôleurs étaient grévistes, des taux en recul par rapport à mardi (49,3 % chez les conducteurs), jour du réveillon de Noël et précédent jour de publication de ces taux. Le taux des aiguilleurs en grève a légèrement augmenté, à 13,8 %.
['▁Au', '▁22', 'e', '▁jour', '▁consécutif', '▁de', '▁la', '▁grève', '▁recon', 'duc', 't', 'ible', '▁contre', '▁la', '▁réforme', '▁des', '▁retraites', '▁(', 'soit', '▁autant', '▁de', '▁jours', '▁que', '▁le', '▁long', '▁conflit', '▁de', '▁1995', '),', '▁le', '▁taux', '▁de', '▁grévistes', '▁à', '▁la', '▁SNCF', '▁a', '▁légèrement', '▁baissé', '▁à', '▁', '9,6', '▁%', '▁j